# Run WRFHydro model on HPC resources using CyberGIS-Compute V2

This notebook demonstrates how to prepare a WRFHydro model on CyberGIS-Jupyter for Water (CJW) for execution on a supported High-Performance Computing (HPC) resource via the CyberGIS-Compute service. First-time users are highly encouraged to go through the [NCAR WRFHydro Hands-on Training on CJW](https://www.hydroshare.org/resource/d2c6618090f34ee898e005969b99cf90/) to get familiar WRFHydro model basics including compilation of source code, preparation of forcing data and typical model configurations. This notebook will not cover those topics and assume users already have hands-on experience with local model runs.

CyberGIS-Compute is a CyberGIS-enabled web service sits between CJW and HPC resources. It acts as a middleman that takes user requests (eg. submission of a model) originated from CJW, carries out the actual job submission of model on the target HPC resource, monitors job status, and retrieves outputs when the model execution has completed. The functionality of CyberGIS-Compute is exposed as a series of REST APIs. A Python client, [CyberGIS-Compute SDK](https://github.com/cybergis/cybergis-compute-python-sdk), has been developed for use in the CJW environment that provides a simple GUI to guide users through the job submission process. Prior to job submission, model configuration and input data should be prepared and arranged in a certain way that meets the model-specific requirements, which vary by models and their implementation in CyberGIS-Compute. We will walk through the requirements for WRFHydro below.

Current support for WRFHydro in CyberGIS-Compute is compatible with WRFHydro 5.x official releases from the NCAR repo (https://github.com/NCAR/wrf_hydro_nwm_public), and requires users to provide parameters & data in 3 categories: **compile-time configuration, run-time configuration, and model data**. The following table lists details for each category.

| Category   |      Parameter/Data     | Required|  Comments | Submission|
|:----------|-------------:|:-----:|------:|--:|
| Compile-Time |  Model_Version (string) | Y |release, tag, branch, commit id from [WRFHydro official repo](https://github.com/NCAR/wrf_hydro_nwm_public) | GUI or API |
| Compile-Time |   "setEnvar.sh" (file)          | Y |[example "setEnvar.sh"](https://github.com/NCAR/wrf_hydro_nwm_public/blob/refs/tags/v5.2.0/trunk/NDHMS/template/setEnvar.sh)  | Upload |
| Compile-Time |   LSM_Type  (string)      |   Y |"NoahMP" (default) or "Noah" | GUI or API |
| Run-Time |  "namelist.hrldas" (file) |   Y | [example "namelist.hrldas" for LSM_Type="NoahMP"](https://github.com/NCAR/wrf_hydro_nwm_public/blob/refs/tags/v5.2.0/trunk/NDHMS/template/NoahMP/namelist.hrldas) | Upload|
| Run-Time |  "hydro.namelist" (file) |   Y | [example "hydro.namelist"](https://github.com/NCAR/wrf_hydro_nwm_public/blob/refs/tags/v5.2.0/trunk/NDHMS/template/HYDRO/hydro.namelist) |Upload|
| Model Data | "DOMAIN" (folder)|  Y | contain domain files | Upload |
| Model Data | "FORCING" (folder) |  Y | contain forcing files | Upload|
| Model Data | "RESTART" (folder)|  N | contain restart files | Upload|

Since WRFHydro codebase has been under active development, "mix-match" parameters or data from different releases may cause tricky issues or model failure. For example, the file 'namelist.hrldas' from version A may not work with WRFHydro version B. **It is user's responbisibity to make sure the parameter and data provided are compatible with the chosen Model_Version.**

We will use the official testcase (Croton, NY) shipped with WRFHydro v5.2.0 release (https://github.com/NCAR/wrf_hydro_nwm_public/releases/tag/v5.2.0) to walk you through.

![WFHydro Official Release v5.2.0](statics/release_v520.png)
FYI: The support for WRFHydro model in CyberGIS-Compute is added through the "community contribution" appoarch (introduced in CyberGIS-Compute V2), and the implementation can be accessible at https://github.com/cybergis/cybergis-compute-v2-wrfhydro. This work is based on a close collabration between the [CyberGIS Center](https://cybergis.illinois.edu/) at UIUC and [Dr. Ayman Nassar](https://uwrl.usu.edu/people/research-staff/nassar-ayman) at Utah Water Research Laboratory, USU under the [HydroShare](https://www.hydroshare.org/) project.

## Setup workspace

In [ ]:
scratch_folder = "./scratch"
upload_folder = "./upload2hpc"

In [ ]:
!rm -rf {scratch_folder} && mkdir -p {scratch_folder}
!rm -rf {upload_folder} && mkdir -p {upload_folder}

## Compile-Time configuration (Model_Version) & Retrieve WRFHydro Codebase v5.2.0

In [ ]:
!rm -rf {scratch_folder}/wrf_hydro_nwm_public
!git clone -b v5.2.0 https://github.com/NCAR/wrf_hydro_nwm_public.git {scratch_folder}/wrf_hydro_nwm_public

## Compile-Time configuration ("setEnvar.sh")

In [ ]:
!cp -f {scratch_folder}/wrf_hydro_nwm_public/trunk/NDHMS/template/setEnvar.sh {upload_folder}
!ls {upload_folder} -l

In [ ]:
!sed -i 's/HYDRO_D=0/HYDRO_D=1/'  {upload_folder}/setEnvar.sh
!sed -i 's/SPATIAL_SOIL=0/SPATIAL_SOIL=1/' {upload_folder}/setEnvar.sh
!cat {upload_folder}/setEnvar.sh

## Retrieve test case (Cronton,NY)

In [ ]:
!rm -rf {scratch_folder}/*.tar.gz {scratch_folder}/example_case
!wget https://github.com/NCAR/wrf_hydro_nwm_public/releases/download/v5.2.0/croton_NY_training_example_v5.2.tar.gz -P {scratch_folder}
!cd {scratch_folder} && tar xzf croton_NY_training_example_v5.2.tar.gz

## Run-Time configurations ("namelist.hrldas", "hydro.namelist") and Model Data ("Domain", "RESTART")

In [ ]:
!ls {scratch_folder}/example_case/Gridded -l

In [ ]:
!cp -rf {scratch_folder}/example_case/Gridded/* {upload_folder}
!ls {upload_folder} -l

## Model Data ("Forcings")

In [ ]:
!cp -rf {scratch_folder}/example_case/FORCING {upload_folder}
!ls {upload_folder} -l

## Job Submission

In [ ]:
from cybergis_compute_client import CyberGISCompute
cybergis = CyberGISCompute(url="cgjobsup-dev.cigi.illinois.edu", isJupyter=True, protocol="HTTPS", port=443, suffix="v2")

In [ ]:
cybergis.create_job_by_ui(defaultJob="wrfhydro-5.x", defaultDataFolder=upload_folder)

## Check out model outputs

In [ ]:
!echo {cybergis.recentDownloadPath}/Simulation

In [ ]:
!ls {cybergis.recentDownloadPath}/Simulation

## Visualization

In [ ]:
import xarray as xr
chanobs = xr.open_mfdataset('{}/Simulation/*CHANOBS*'.format(cybergis.recentDownloadPath),
                            combine='by_coords')

In [ ]:
chanobs.sel(feature_id = 2).streamflow.plot()

#### Done